### VECTOR Creator

In [2]:
# FILE LOADER
from langchain.document_loaders import PyPDFLoader

file_path = "/Users/rehan/openai/textbook_pdfs/chemistry10.pdf"

def file_loader(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    return pages

pages = file_loader(file_path)

In [4]:
# LOAD FAISS DB / VECTOR DB
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


def create_embedding(pages):
    vector_store = FAISS.from_documents(pages, OpenAIEmbeddings(model="text-embedding-3-large"))
    return vector_store

def load_embedding(name):
    vector_store = FAISS.load_local(name, OpenAIEmbeddings(model="text-embedding-3-large"))
    return vector_store

def save_embedding(name, vector_store):
    vector_store.save_local(name)

vector_store = create_embedding(pages)
save_embedding("chemistry10", vector_store)

### Similarity Search

In [14]:
# SIMILARITY SEARCH
# FIND THE MOST MATCHING PAGE

def search(question, vector_store, k=2):
    docs = vector_store.similarity_search(question, k)
    context = ""
    for doc in docs:
        context += doc.page_content
        # print(str(doc.metadata["page"]) + ":", doc.page_content[:10000])

    return context

### Template Creator

In [15]:
template = """
{instruction} Please provide a brief response consisting of {number} distinct points.

Context:
{context}

Question: {question}
"""

In [16]:
# INSTRUCTION
instruction = """As a student studying Class 10 History following the CBSE curriculum, I kindly request your assistance in answering a set of questions related to the Class 10 history exam. In order to ensure accuracy, please base your responses solely on the information presented in the context. It is crucial that you strictly adhere to the precise wording used in the context and refrain from incorporating any external knowledge into your answers."""

In [17]:
# USER QUESTION
#question = "Explain the meaning and notion of ‘Swaraj’ as perceived by the plantation workers. How did they respond to the call of ‘the Non – Cooperation movement’?"

#number = "5"

### Token Counter

In [18]:
import tiktoken


def num_tokens(message, model="gpt-3.5-turbo-0613"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(message))
    return num_tokens

In [19]:
def update_log(instruction, question, context, answer):
    with open('logs.txt', 'a') as file:
        file.write(f"Instruction: {instruction}\n")
        file.write(f"Question: {question}\n")
        file.write(f"Context: {context}\n")
        file.write(f"Answer: {answer}\n")
        file.write("--------------------------------------------------------\n")

### Run langchang

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# vector_store = create_embedding(pages)
# save_embedding("science10", vector_store)
vector_store = load_embedding("history10")

question = input("Question: ")
number = str(input("Marks: "))

context_text = search(question, vector_store)
prompt_template = ChatPromptTemplate.from_template(template)
prompt = prompt_template.format(instruction=instruction, number=number, context=context_text, question=question)
# print(prompt)

model = ChatOpenAI(model_name="gpt-3.5-turbo-1106")
response_text = model.predict(prompt)
formatted_response = f"Response:\n{response_text}"

print("Question:\n" + question)
print(formatted_response)

print("Input tokens: " + str(num_tokens(prompt)))
print("Output tokens: " + str(num_tokens(response_text)))

update_log(instruction, question, context_text, response_text)

Question:
what is swaraj
Response:
1. Swaraj was interpreted by tribal peasants and plantation workers in their own ways, based on their understanding of Mahatma Gandhi's message.
2. For the tribal peasants, swaraj meant being able to graze their cattle, collect fuel wood and fruits from the forests, and retain their traditional rights.
3. Alluri Sitaram Raju, a leader of the Gudem Hills rebellion, believed that swaraj could only be achieved through the use of force, not non-violence.
4. For plantation workers in Assam, swaraj meant the right to move freely in and out of the confined space in which they were enclosed, and retaining a link with their villages.
5. The visions of swaraj held by these movements were not defined by the Congress programme, but they emotionally related to an all-India agitation and identified with a movement that went beyond the limits of their immediate locality.
Input tokens: 1008
Output tokens: 186
